In [1]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import dash_daq as daq
import plotly.express as px
import numpy as np
import random
import plotly.graph_objects as go

In [18]:
df = pd.read_parquet('./CKW_open_data/part-00000-tid-1610874990929950894-68fe3bd4-944f-4060-bf27-a0fc69f4cff2-626-1-c000.snappy.parquet')
# Force value_kwh to be float
df['value_kwh'] = df['value_kwh'].astype(float)
# drop count
df = df.drop(columns=['count'])
df

,id,timestamp,value_kwh
0,031739d5539bdbd1fd0e41b688760059,2022-03-04 20:30:00,0.476
1,031739d5539bdbd1fd0e41b688760059,2022-03-07 10:45:00,0.485
2,031739d5539bdbd1fd0e41b688760059,2022-03-11 23:45:00,0.492
3,031739d5539bdbd1fd0e41b688760059,2022-03-13 20:15:00,0.497
4,031739d5539bdbd1fd0e41b688760059,2022-03-14 14:00:00,0.018
...,...,...,...
8011155,fc665fd292a62baf0ca97da6992117c1,2023-08-24 08:30:00,0.002
8011156,fc665fd292a62baf0ca97da6992117c1,2023-08-26 02:45:00,0.002
8011157,fc665fd292a62baf0ca97da6992117c1,2023-08-26 21:00:00,0.005
8011158,fc665fd292a62baf0ca97da6992117c1,2023-08-27 09:15:00,0.005


In [19]:
# Replace IDs with 1-nunique IDs
df['id'] = df['id'].astype('category').cat.codes
df['id'] = df['id'] + 1
df

,id,timestamp,value_kwh
0,2,2022-03-04 20:30:00,0.476
1,2,2022-03-07 10:45:00,0.485
2,2,2022-03-11 23:45:00,0.492
3,2,2022-03-13 20:15:00,0.497
4,2,2022-03-14 14:00:00,0.018
...,...,...,...
8011155,150,2023-08-24 08:30:00,0.002
8011156,150,2023-08-26 02:45:00,0.002
8011157,150,2023-08-26 21:00:00,0.005
8011158,150,2023-08-27 09:15:00,0.005


In [20]:
df.to_parquet('./CKW_open_data/processed_data.parquet')

In [52]:
app = Dash(__name__)

def hex_to_rgba(hex_color, opacity):
    hex_color = hex_color.lstrip('#')
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    opacity = max(0, min(1, opacity))
    return f'rgba({r}, {g}, {b}, {opacity})'

min_date = df['timestamp'].min()
max_date = df['timestamp'].max()

app.layout = html.Div([
    html.H1(children='A look at smart meters',
            style={'textAlign': 'center', 'font-size': '40px', 'font-family': 'sans-serif'}),
    html.H2(children='Select a smart meters:',
            style={'textAlign': 'center', 'font-size': '30px', 'font-family': 'sans-serif'}),
    html.Div([
        html.Div([
            html.Label('Date range:', style={'width': '90%', 'margin': 'auto', 'font-size': '20px', 'font-family': 'sans-serif'}),
            dcc.DatePickerRange(id='date-picker',
                                min_date_allowed=min_date,
                                max_date_allowed=max_date,
                                start_date=min_date,
                                end_date=min_date+pd.Timedelta(days=14),
                                style={'width': '90%', 'margin': 'auto', 'font-size': '20px', 'font-family': 'sans-serif', 'display': 'block'}),
        ], style={'width': '30%', 'margin': 'auto', 'display': 'inline-block'}),
        html.Div([
            html.Label('Meter IDs:', style={'width': '90%', 'margin': 'auto', 'font-size': '20px', 'font-family': 'sans-serif'}),
            dcc.Dropdown(options=[{'label': meter_id, 'value': meter_id} for meter_id in df['id'].unique()],
                         id='meter-dropdown',
                         multi=True,
                         value=[df['id'].unique()[0]],
                         style={'width': '90%', 'margin': 'auto', 'font-size': '20px', 'font-family': 'sans-serif', 'display': 'block'}),
        ], style={'width': '30%', 'margin': 'auto', 'display': 'inline-block'}),
        html.Div([
            html.Label('Addons:', style={'width': '90%', 'margin': 'auto', 'font-size': '20px', 'font-family': 'sans-serif'}),
            dcc.Dropdown(options=[{'label': 'Confidence interval', 'value': 'confidence-interval'}],
                         id='addon-dropdown',
                         multi=True,
                         value=[],
                         style={'width': '90%', 'margin': 'auto', 'font-size': '20px', 'font-family': 'sans-serif', 'display': 'block'}),
        ], style={'width': '30%', 'margin': 'auto', 'display': 'inline-block'})
    ], style={'margin-top': '20px', 'margin-bottom': '20px', 'width': '95%', 'text-align': 'center', 'display': 'block', 'margin': 'auto'}),
    dcc.Graph(id='meter-graph',
              style={'width': '100%', 'margin': 'auto', 'display': 'block', 'margin-top': '25px'})
])

@app.callback(Output('meter-graph', 'figure'),
              [Input('meter-dropdown', 'value'),
               Input('date-picker', 'start_date'),
               Input('date-picker', 'end_date'),
               Input('addon-dropdown', 'value')])

def update_graph(selected_meters, start_date, end_date, addons):
    fig = px.line()
    for i, meter_id in enumerate(selected_meters):
        df_meter = df[df['id'] == meter_id]
        df_meter = df_meter[(df_meter['timestamp'] >= start_date) & (df_meter['timestamp'] <= end_date)]
        df_meter = df_meter.sort_values('timestamp')
        
        main_trace = go.Scatter(x=df_meter['timestamp'], y=df_meter['value_kwh'], mode='lines', name=f'Meter {i+1}')
        fig.add_trace(main_trace)

        # Removed confidence interval for now as in the final product, there would only be one meter using a CI at a time
    
    fig.update_layout(title='Energy consumption for selected meters')
    fig.update_xaxes(title='Time')
    fig.update_yaxes(title='Energy consumption (kWh)')
    
    return fig

app.run_server(debug=True)